In [ ]:
import uproot
import awkward as ak
import numpy as np
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema, DelphesSchema
from coffea import hist, processor
from yahist import Hist1D, Hist2D

In [ ]:
from tools.helpers import *

In [ ]:
events_150 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/2HDMA_bb-gg_YY_XX_10_v4/2HDMa_bb_1500_150_10_delphes_ntuple.root:myana',
        treepath='mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

events_250 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/2HDMA_bb-gg_YY_XX_10_v4/2HDMa_bb_1500_250_10_delphes_ntuple.root',
        treepath='myana/mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

events_350 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/2HDMA_bb-gg_YY_XX_10_v4/2HDMa_bb_1500_350_10_delphes_ntuple.root',
        treepath='myana/mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

events_500 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/2HDMA_bb-gg_YY_XX_10_v4/2HDMa_bb_1500_500_10_delphes_ntuple.root',
        treepath='myana/mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()   

events_750 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/2HDMA_bb-gg_YY_XX_10_v4/2HDMa_bb_1500_750_10_delphes_ntuple.root',
        treepath='myana/mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

In [ ]:
events_bb_1750_750 = NanoEventsFactory.from_root(
        '/nfs-7/userdata/ewallace/Hbb_MET/2HDMA_bb-gg_YY_XX_10_v4/2HDMa_gg_2000_750_10_delphes_ntuple.root',
        treepath='myana/mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

In [ ]:
len(events_bb_1750_750)

In [ ]:
from yaml import Loader, Dumper
import yaml
    
with open('../data/samples.yaml', 'r') as f:
    samples = yaml.load(f, Loader = Loader)

fileset = {
            #'TT_TuneCUETP8M2T4_14TeV-powheg-pythia8_200PU': samples['TT_TuneCUETP8M2T4_14TeV-powheg-pythia8_200PU']['skim'],
            #'ZJetsToNuNu_HT-100To200_14TeV-madgraph_200PU': samples['ZJetsToNuNu_HT-100To200_14TeV-madgraph_200PU']['skim'],
            'ZJetsToNuNu_HT-200To400_14TeV-madgraph_200PU': samples['ZJetsToNuNu_HT-200To400_14TeV-madgraph_200PU']['skim'],
            'ZJetsToNuNu_HT-400To600_14TeV-madgraph_200PU': samples['ZJetsToNuNu_HT-400To600_14TeV-madgraph_200PU']['skim'],
            'ZJetsToNuNu_HT-600To800_14TeV-madgraph_200PU': samples['ZJetsToNuNu_HT-600To800_14TeV-madgraph_200PU']['skim'],
            'ZJetsToNuNu_HT-800To1200_14TeV-madgraph_200PU': samples['ZJetsToNuNu_HT-800To1200_14TeV-madgraph_200PU']['skim'],
            'ZJetsToNuNu_HT-1200To2500_14TeV-madgraph_200PU': samples['ZJetsToNuNu_HT-1200To2500_14TeV-madgraph_200PU']['skim'],
            #'W0JetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU': samples['W0JetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['skim'],
            #'W1JetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU': samples['W1JetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['skim'],
            #'W2JetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU': samples['W2JetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['skim'],
            #'W3JetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU': samples['W3JetsToLNu_TuneCUETP8M1_14TeV-madgraphMLM-pythia8_200PU']['skim'],
            #'2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_150_MH2_1500_MHC_1500': samples['2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_150_MH2_1500_MHC_1500']['ntuples'],
            #'2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500': samples['2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_250_MH2_1500_MHC_1500']['ntuples'],
            #'2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_350_MH2_1500_MHC_1500': samples['2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_350_MH2_1500_MHC_1500']['ntuples'],
            #'2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_500_MH2_1500_MHC_1500': samples['2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_500_MH2_1500_MHC_1500']['ntuples'],
            #'2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500': samples['2HDMa_bb_sinp_0.35_tanb_1.0_mXd_10_MH3_1500_MH4_750_MH2_1500_MHC_1500']['ntuples'],
        }

meta_accumulator = {}
for sample in fileset:
    if sample not in meta_accumulator:
        meta_accumulator.update({sample: processor.defaultdict_accumulator(int)})
    for f in fileset[sample]:
        meta_accumulator.update({f: processor.defaultdict_accumulator(int)})

In [ ]:
events_250 = NanoEventsFactory.from_root(
        samples['ZJetsToNuNu_HT-200To400_14TeV-madgraph_200PU']['skim'][0],
        treepath='mytree',
        schemaclass = BaseSchema,
        #entry_stop = n_max,
    ).events()

In [ ]:
electron = get_four_vec_fromPtEtaPhiM(
            None,
            pt = events_250.elec_pt,
            eta = events_250.elec_eta,
            phi = events_250.elec_phi,
            M = events_250.elec_mass,
            copy = False,
        )
electron['id'] = events_250.elec_idpass  # > 0 should be loose
electron['iso'] = events_250.elec_isopass
electron['charge'] = events_250.elec_charge

ele_l = electron[((electron['id']>0)&(electron['iso']>0)&(electron.pt>10)&(np.abs(electron.eta)<3))]
        
        #muons
        
muon = get_four_vec_fromPtEtaPhiM(
            None,
            pt = events_250.muon_pt,
            eta = events_250.muon_eta,
            phi = events_250.muon_phi,
            M = events_250.muon_mass,
            copy = False,
        )
muon['id'] = events_250.muon_idpass  # > 0 should be loose
muon['iso'] = events_250.muon_isopass
muon['charge'] = events_250.muon_charge

muon_l = muon[((muon['id']>0)&(muon['iso']>0)&(muon.pt>4)&(np.abs(muon.eta)<2.8))]
        
#taus
        
tau = get_four_vec_fromPtEtaPhiM(
            None,
            pt = events_250.tau_pt,
            eta = events_250.tau_eta,
            phi = events_250.tau_phi,
            M = events_250.tau_mass,
            copy = False,
        )
tau['iso'] = events_250.tau_isopass   # > 0 should be loose
tau['charge'] = events_250.tau_charge

tau_l = tau[((tau['iso']>0)&(tau.pt>30)&(np.abs(tau.eta)<3))]
        
#photons
        
gamma = get_four_vec_fromPtEtaPhiM(
            None,
            pt = events_250.gamma_pt,
            eta = events_250.gamma_eta,
            phi = events_250.gamma_phi,
            M = events_250.gamma_mass,
            copy = False,
        )
gamma['id'] = events_250.gamma_idpass  # > 0 should be loose
gamma['iso'] = events_250.gamma_isopass

gamma_l = gamma[((gamma['id']>0)&(gamma['iso']>0)&(gamma.pt>20)&(np.abs(gamma.eta)<3))]


jet = get_four_vec_fromPtEtaPhiM(
            None,
            pt = events_250.jetpuppi_pt,
            eta = events_250.jetpuppi_eta,
            phi = events_250.jetpuppi_phi,
            M = events_250.jetpuppi_mass,
            copy = False,
        )
jet['id'] = events_250.jetpuppi_idpass
jet['btag'] = events_250.jetpuppi_btag
        
        
#follow Delphes recommendations
jet = jet[jet.pt > 30]
jet = jet[jet.id > 0]
jet = jet[np.abs(jet.eta) < 3] #eta within tracker range
        
jet = jet[~match(jet, electron, deltaRCut=0.4)] #remove electron overlap
jet = jet[~match(jet, muon, deltaRCut=0.4)] #remove muon overlap

btag = jet[jet.btag>0] #loose wp for now
        
#fatjets

# Need FatJets and GenParts
# FatJets start at pt>200 and go all the way to eta 3.x
# This should be fine?
# Objects are defined here: https://twiki.cern.ch/twiki/bin/view/CMS/DelphesInstructions
# Maybe restrict abs(eta) to 2.8 or 3 (whatever the tracker acceptance of PhaseII CMS is)
fatjet = get_four_vec_fromPtEtaPhiM(
            None,
            pt = events_250.fatjet_pt,
            eta = events_250.fatjet_eta,
            phi = events_250.fatjet_phi,
            M = events_250.fatjet_msoftdrop,        #Using softdrop from now on
            copy = False,
        )
        
#fatjet['m'] = events.fatjet_mass
fatjet['tau1'] = events_250.fatjet_tau1
fatjet['tau2'] = events_250.fatjet_tau2
fatjet['tau3'] = events_250.fatjet_tau3
fatjet['tau4'] = events_250.fatjet_tau4
        
#eta within tracker range
fatjet = fatjet[np.abs(fatjet.eta) < 3]
fatjet = fatjet[ak.argsort(fatjet.pt, ascending=False)]
        
fatjet_on_h = fatjet[np.abs(fatjet.mass-125)<25]
on_h = (ak.num(fatjet_on_h) > 0)
        
lead_fatjet = fatjet[:,0:1]
#sublead_fatjet = fatjet[:,1:2] 
        
lead_fatjets = fatjet[:,0:2]
difatjet = choose(lead_fatjets, 2)
dphiDiFatJet = np.arccos(np.cos(difatjet['0'].phi-difatjet['1'].phi))
        
tau21 = np.divide(lead_fatjet.tau2, lead_fatjet.tau1)
        
extrajet  = jet[~match(jet, fatjet, deltaRCut=1.2)] # remove AK4 jets that overlap with AK8 jets
extrabtag = extrajet[extrajet.btag>0] #loose wp for now]
extrajet = extrajet[ak.argsort(extrajet.pt, ascending=False)]
extrabtag = extrabtag[ak.argsort(extrabtag.pt, ascending=False)]
lead_extrajet = extrajet[:,0:1]
lead_extrabtag = extrabtag[:,0:1]

met_pt = ak.flatten(events_250.metpuppi_pt)
met_phi = ak.flatten(events_250.metpuppi_phi)

In [ ]:
lead_fatjets = fatjet[:,0:2]
difatjet = choose(lead_fatjets, 2)
dphiDiFatJet = np.arccos(np.cos(difatjet['0'].phi-difatjet['1'].phi))

In [ ]:
extrajet  = jet[~match(jet, fatjet, deltaRCut=1.2)] # remove AK4 jets that overlap with AK8 jets
extrabtag = extrajet[extrajet.btag>0] #loose wp for now]
extrajet = extrajet[ak.argsort(extrajet.pt, ascending=False)]
extrabtag = extrabtag[ak.argsort(extrabtag.pt, ascending=False)]
lead_extrajet = extrajet[:,0:1]
lead_extrabtag = extrabtag[:,0:1]

In [ ]:
dphileadextrajet = delta_phi_alt_paf(lead_fatjet, lead_extrajet)

In [ ]:
lead_fatjet.phi

In [ ]:
met_phi

In [ ]:
run2_to_delphes = {
            'ZJetsToNuNu_HT-200To400_14TeV-madgraph_200PU': 'ZJetsToNuNu_HT-200To400_TuneCP5_13TeV-madgraphMLM-pythia8',
            'ZJetsToNuNu_HT-400To600_14TeV-madgraph_200PU': 'ZJetsToNuNu_HT-400To600_TuneCP5_13TeV-madgraphMLM-pythia8',
            'ZJetsToNuNu_HT-600To800_14TeV-madgraph_200PU': 'ZJetsToNuNu_HT-600To800_TuneCP5_13TeV-madgraphMLM-pythia8',
            'ZJetsToNuNu_HT-800To1200_14TeV-madgraph_200PU': 'ZJetsToNuNu_HT-800To1200_TuneCP5_13TeV-madgraphMLM-pythia8',
            'ZJetsToNuNu_HT-1200To2500_14TeV-madgraph_200PU': 'ZJetsToNuNu_HT-1200To2500_TuneCP5_13TeV-madgraphMLM-pythia8',
}

effs = {}
for s in fileset.keys():
    effs[s] = {}
    for b in ['0b', '1b', '2b', '1h']:
        effs[s][b] = Hist2D.from_json(os.path.expandvars("../data/htag/eff_%s_%s.json"%(run2_to_delphes[s],b)))

In [ ]:
gen = get_four_vec_fromPtEtaPhiM(
            None,
            pt = events_250.genpart_pt,
            eta = events_250.genpart_eta,
            phi = events_250.genpart_phi,
            M = events_250.genpart_mass,
            copy = False,
        )
gen['pdgId'] = events_250.genpart_pid
gen['status'] = events_250.genpart_status

higgs = gen[((abs(gen.pdgId)==25)&(gen.status==62))]

bquark = gen[((abs(gen.pdgId)==5)&(gen.status==71))]
        
nb_in_fat = match_count(fatjet, bquark, deltaRCut=0.8)
nhiggs_in_fat = match_count(fatjet, higgs, deltaRCut=0.8)
zerohiggs = (nhiggs_in_fat==0)
onehiggs = (nhiggs_in_fat==1)
        
zerob = ((nb_in_fat==0) & (zerohiggs))  # verified to work!
oneb  = ((nb_in_fat==1) & (zerohiggs))  # verified to work!
twob  = ((nb_in_fat>=2) & (zerohiggs))  # verified to work!
        
w_0b = get_weight(effs['ZJetsToNuNu_HT-200To400_14TeV-madgraph_200PU']['0b'], fatjet.pt, fatjet.eta)
w_1b = get_weight(effs['ZJetsToNuNu_HT-200To400_14TeV-madgraph_200PU']['1b'], fatjet.pt, fatjet.eta)
w_2b = get_weight(effs['ZJetsToNuNu_HT-200To400_14TeV-madgraph_200PU']['2b'], fatjet.pt, fatjet.eta)
w_1h = get_weight(effs['ZJetsToNuNu_HT-200To400_14TeV-madgraph_200PU']['1h'], fatjet.pt, fatjet.eta)
        
w_all = w_0b * zerob + w_1b * oneb + w_2b * twob # + w_1h * onehiggs  # this should work
if not np.isnan(sum(sum(effs['ZJetsToNuNu_HT-200To400_14TeV-madgraph_200PU']['1h'].counts))):
#        np.isnan(sum(ak.flatten(w_1h * onehiggs))):
    w_all = w_all + w_1h * onehiggs

In [ ]:
ak.to_numpy(min_dphiFatJetMet4[baseline])

In [ ]:
np.nan_to_num(1-ak.prod(1-w_all[baseline], axis=1), 0)

In [ ]:
fatjet_on_h = fatjet[np.abs(fatjet.mass-125)<25]
on_h = (ak.num(fatjet_on_h) > 0)
mt_fj_met = mt(fatjet_on_h.pt, fatjet_on_h.phi, met_pt, met_phi)
min_mt_fj_met = ak.min(mt_fj_met, axis=1, mask_identity=False)
min_dphiFatJetMet4 = ak.min(np.arccos(np.cos(fatjet[:,:4].phi-met_phi)), axis=1)
min_dphiJetMetAll = ak.min(np.arccos(np.cos(jet.phi-met_phi)), axis=1)

In [ ]:
min_dphiFatJetMet4[min_dphiFatJetMet4>4]

In [ ]:
min_dphiJetMetAll[min_dphiJetMetAll>4]

In [ ]:
ele_sel = (ak.num(ele_l)==0)
mu_sel = ((ak.num(muon_l)==0) & ele_sel)
tau_sel = ((ak.num(tau_l)==0) & mu_sel)
met_sel = (tau_sel & (met_pt>200))
baseline = ((ak.num(fatjet)>0) & met_sel)
min_dphiFatJetMet4_sel = ((min_dphiFatJetMet4 > 0.5) & baseline)
dphiDiFatJet_sel = (ak.all(dphiDiFatJet < 2.5, axis=1) & min_dphiFatJetMet4_sel)
on_h_sel = (on_h & dphiDiFatJet_sel)
min_mt = ((min_mt_fj_met > 200) & on_h_sel)

In [ ]:
min_dphiFatJetMet4_sel = ak.to_numpy((min_dphiFatJetMet4 > 0.5) & baseline)

In [ ]:
min_dphiFatJetMet4_sel